<h1 align="center"><font color="yellow">Pytorch: Datasets</font></h1>

<font color="yellow">Data Scientist.: PhD.Eddy Giusepe Chirinos Isidro</font>

<font color="orange">Neste script vamos a estudar como o Pytorch define nossos datasets.</font>

In [ ]:
%conda install requests,matplotlib --yes

In [1]:
%load_ext watermark 
%watermark -v -p numpy,pandas,matplotlib,requests,torch

Python implementation: CPython
Python version       : 3.9.13
IPython version      : 8.13.2

numpy     : 1.24.3
pandas    : 2.0.1
matplotlib: 3.7.1
requests  : 2.31.0
torch     : 2.0.1



# Iterando Tensores

`Lembrar:`

<font color="orange">O conjunto de MNIST é um conjunto de Dados muito utilizado na área de `Visão Computacional`. As principais características são:</font>

* `Imagens em escala de cinza:` Cada imagem do MNIST tem dimensões de `28x28 pixels` e é representada em escala de cinza. Isso significa que cada pixel pode ter um valor entre `0 e 255`, onde <font color="orange">0 representa o preto</font> e <font color="orange">255 representa o branco</font>.

* `Dígitos de 0 a 9:` O conjunto de dados MNIST contém imagens de dígitos manuscritos de `0 a 9`. Cada imagem é rotulada com o dígito correspondente.

* `Conjunto de treinamento e teste:` O conjunto de dados MNIST é dividido em dois conjuntos: um conjunto de treinamento e um conjunto de teste. O conjunto de treinamento contém `60.000` imagens, enquanto o conjunto de teste contém `10.000` imagens.

* `Equilíbrio de classes:` O MNIST é equilibrado em relação às classes, o que significa que cada dígito (0 a 9) tem uma quantidade semelhante de exemplos no conjunto de dados.


In [2]:
import torch
from sklearn.datasets import fetch_openml
import numpy as np

# Descarregando nosso Dataset
mnist = fetch_openml('mnist_784', version=1)
X, Y = mnist["data"], mnist["target"]

# Normalizando:
X_train, X_test, y_train, y_test = X[:60000] / 255., X[60000:] / 255., Y[:60000].astype(int), Y[60000:].astype(int)


X_t = torch.from_numpy(X_train.values).float().cuda()
Y_t = torch.from_numpy(y_train.values).long().cuda()


/home/eddygiusepe/miniconda3/envs/LightningAI/lib/python3.9/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
D_in, H, D_out = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
).to("cuda")

In [4]:
from sklearn.metrics import accuracy_score

def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)

def evaluate(x):
    model.eval()
    y_pred = model(x)
    y_probas = softmax(y_pred)
    return torch.argmax(y_probas, axis=1)

In [5]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)


epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1): 
    
    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())
    
    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()
    
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")
        
y_pred = evaluate(torch.from_numpy(X_test.values).float().cuda())

print("")
print("\033[93mA accuracy é: \033[0m")
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 1.80345
Epoch 20/100 Loss 1.53021
Epoch 30/100 Loss 1.25949
Epoch 40/100 Loss 1.07717
Epoch 50/100 Loss 0.95154
Epoch 60/100 Loss 0.85129
Epoch 70/100 Loss 0.77467
Epoch 80/100 Loss 0.71532
Epoch 90/100 Loss 0.66693
Epoch 100/100 Loss 0.62649

A accuracy é: 


0.9302

# Iterando por Batches

<font color="orange">Na implementação anterior estamos Otimizando nosso modelo com o Algoritmo de `batch gradient descent`, na qual utilizamos todos nossos Dados em cada passo de Otimização. No entanto, um algoritmo que pode convergir mais rápido (e única opção se nosso dataset é tão grande que não cabe em memória) é o de `mini-batch gradient descent`.</font>

In [ ]:
D_in, H, D_out = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
).to("cuda")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 100
batch_size = 100
log_each = 1
l = []
model.train()
batches = len(X_t) // batch_size
for e in range(1, epochs+1): 
    
    _l = []
    # Iteramos por batches
    for b in range(batches):
        x_b = X_t[b*batch_size:(b+1)*batch_size]
        y_b = Y_t[b*batch_size:(b+1)*batch_size]
        
        # forward
        y_pred = model(x_b)

        # loss
        loss = criterion(y_pred, y_b)
        _l.append(loss.item())

        # ponemos a cero los gradientes
        optimizer.zero_grad()

        # Backprop (calculamos todos los gradientes automáticamente)
        loss.backward()

        # update de los pesos
        optimizer.step()
    
    l.append(np.mean(_l))
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")
        
y_pred = evaluate(torch.from_numpy(X_test.values).float().cuda())

print("")
print("\033[93mA accuracy é: \033[0m")
accuracy_score(y_test, y_pred.cpu().numpy())

Esta implementação é correta e funcional, dependendo de nossos dados pode chegar a ser mais complexa (<font color="orange">`Por exemplo:` se precisarmos carregar muitas imagens à quais queremos aplicar Transformações, juntar batches, etc</font>) Ademais, é comum re-utilizar a lógica para carregar nossos dados não só para treinar a rede, senão para gerar predições. Este fato motiva o uso das Classes especiais que `Pytorch` nos oferece para isso. 

# A classe Dataset

Começamos estudando a classe `Dataset`. Esta classe herda da classe pai `torch.utils.data.Dataset` e temos que definir, como mínimo, três funções: 

- `__init__`: o construtor
- `__len__`: retorna o número de amostras no dataset
- `__getitem__`: retorna uma amostra em concreto do dataset

Uma vez definida a classe, ésta pode se usada como se de qualquer iterador se trata-se.

In [12]:
# A classe Dataset, herda da classe `torch.utils.data.Dataset`
class Dataset(torch.utils.data.Dataset):
    # Construtor
    def __init__(self, X, Y):
        self.X = torch.from_numpy(X.values).float().cuda()
        self.Y = torch.from_numpy(Y.values).long().cuda() # .long() --> Passa para Inteiro Longo (para representar comumente Rótulos ou Categorias)
    # Retornamos o número de dados no dataset
    def __len__(self):
        return len(self.X)
    # Retonamos o elemento `ix` del dataset
    def __getitem__(self, ix):
        return self.X[ix], self.Y[ix]
    

Uma vez definida a Classe, podemos instanciar um objeto que podemos usar para Iterar pelos nossos Dados:

In [13]:
dataset = Dataset(X_train, y_train)

len(dataset)

60000

In [17]:
D_in, H, D_out = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
).to("cuda")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 30
batch_size = 100
log_each = 1
l = []
model.train()
batches = len(dataset) // batch_size
for e in range(1, epochs+1): 
    
    _l = []
    # iteramos por batches en el dataset
    for b in range(batches):
        x_b, y_b = dataset[b*batch_size:(b+1)*batch_size]
        
        # forward
        y_pred = model(x_b)

        # loss
        loss = criterion(y_pred, y_b)
        _l.append(loss.item())

        # ponemos a cero los gradientes
        optimizer.zero_grad()

        # Backprop (calculamos todos los gradientes automáticamente)
        loss.backward()

        # update de los pesos
        optimizer.step()
    
    l.append(np.mean(_l))
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")
        
y_pred = evaluate(torch.from_numpy(X_test.values).float().cuda())


print("")
print("\033[93mA accuracy é: \033[0m")
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 1/30 Loss 0.31284
Epoch 2/30 Loss 0.22046
Epoch 3/30 Loss 0.17761
Epoch 4/30 Loss 0.15118
Epoch 5/30 Loss 0.13265
Epoch 6/30 Loss 0.11857
Epoch 7/30 Loss 0.10724
Epoch 8/30 Loss 0.09780
Epoch 9/30 Loss 0.08982
Epoch 10/30 Loss 0.08291
Epoch 11/30 Loss 0.07695
Epoch 12/30 Loss 0.07172
Epoch 13/30 Loss 0.06717
Epoch 14/30 Loss 0.06305
Epoch 15/30 Loss 0.05936
Epoch 16/30 Loss 0.05604
Epoch 17/30 Loss 0.05305
Epoch 18/30 Loss 0.05034
Epoch 19/30 Loss 0.04787
Epoch 20/30 Loss 0.04562
Epoch 21/30 Loss 0.04358
Epoch 22/30 Loss 0.04170
Epoch 23/30 Loss 0.03997
Epoch 24/30 Loss 0.03838
Epoch 25/30 Loss 0.03691
Epoch 26/30 Loss 0.03554
Epoch 27/30 Loss 0.03428
Epoch 28/30 Loss 0.03310
Epoch 29/30 Loss 0.03199
Epoch 30/30 Loss 0.03096

A accuracy é: 


0.9767

Como você pode ver, podemos ITERAR diretamente sobre o objeto `dataset` da mesma maneira que fizemos anteriormente, no entanto `Pytorch` não oferece outro objeto que nos facilte as coisas na hora de Iterar por bacthes. 

# A Classe DataLoader